Get all targets

In [1]:
from __future__ import print_function, division
import os
import cv2
import argparse
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


class Attributes:
    def __init__(self):
        self.batch_size = 32
        self.test_batch_size = 1000
        self.epochs = 100
        self.lr = 1.0
        self.step = 1
        self.gamma = 0.7
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 100

args = Attributes()
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
torch.manual_seed(args.seed)

In [2]:
df = pd.read_csv('data/imet-2020-fgvc7/train.csv')
N_CLASSES = 3474
targets = np.zeros([df.shape[0],3474])
labs = df.iloc[:,1].str.split().to_numpy()
for (i,ls) in enumerate(labs):
    for l in ls:
        targets[(i,int(l))] = 1
np.savetxt('data/targets.txt', targets, fmt='%d')

KeyboardInterrupt: 

In [8]:
# Alternative (load targets)
import numpy as np
targets = np.loadtxt('data/targets.txt', dtype=int)

Grouping (Kmeans)

In [3]:
# Kmeans grouping
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=200, random_state=0).fit(targets)
kmeans.labels_
np.savetxt('data/kmeans200.txt', kmeans.labels_, fmt='%d')
print("data/kmeans200 saved")

data/kmeans200 saved


Grouping (PCA)

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(targets)
new_targets = pca.transform(targets)

PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [11]:
new_targets = pca.transform(targets)
np.savetxt('data/PCA50.txt', new_targets, fmt='%d')
print("data/PCA50 saved")

data/PCA50 saved


In [ ]:
from sklearn.manifold import TSNE
output_embedded = TSNE(n_components=3).fit_transform(targets)


In [ ]:
print(targets.shape)